In [1]:
import plotly.graph_objects as go
import plotly.express as px

In [2]:
def tg(left_point, right_point):
    if left_point[0]==right_point[0]:
        tg=float("inf")*(-left_point[1]+right_point[1])
    else:
        tg=(left_point[1]-right_point[1])/(left_point[0]-right_point[0])
    return tg


In [24]:
class One_Line:
    def __init__ (self,start_point,err):
        self.start_point=start_point
        # self.finish_point=start_point
        self.finish_point=[start_point[0]+1,start_point[1]+1]
        self.err=err
        self.points=[self.start_point,self.finish_point]
        self.s_a,self.s_b=self.err_diapason(start_point)
        self.s_d,self.s_c=self.err_diapason(self.finish_point)
        self.pmin=tg(self.s_a,self.s_c)
        self.pmax=tg(self.s_b,self.s_d)
        self.cvx_bot=[self.s_a,self.s_d]
        self.cvx_top=[self.s_b,self.s_c]
        print(self.pmin, self.pmax)
    def err_diapason(self, point):
        plus_err=[point[0],point[1]+self.err]
        minus_err=[point[0],point[1]-self.err]
        return plus_err,minus_err
    def check_not_outside(self, new_point):
        plus_err,minus_err=self.err_diapason(new_point)
        # print(tg(self.cvx_bot[0],minus_err),self.pmax)
        # print(tg(self.cvx_top[0],plus_err),self.pmin)
        return tg(self.s_b,minus_err)<=self.pmax and tg(self.s_a,plus_err)>=self.pmin
    def show_spline(self):
        step_right=1
        fig=go.Figure()
        for i in self.points:
            fig=self.plot_point(fig,i)
        # fig=self.plot_line(fig,self.points, "points")
        # for i in self.err_array(self.points):
        #     fig=self.plot_line(fig,i, "err")
        fig=self.plot_line(fig,[self.s_a,self.s_c, [self.s_c[0]+step_right,int(self.s_c[1]+step_right*self.pmin)]], "p")
        fig=self.plot_line(fig,[self.s_b,self.s_d, [self.s_d[0]+step_right,int(self.s_d[1]+step_right*self.pmax)]], "p")
        # fig=self.plot_line(fig,[self.s_b,self.s_d], "p")
        fig.show()
        
    def plot_point(self, fig, point):
        plus_err,minus_err=self.err_diapason(point)
        
        # fig=px.scatter(self.points)
        fig=self.plot_line(fig,[plus_err,point,minus_err],"",showlegend=False )
        return fig
        
    
    def plot_line(self, fig, line, name, showlegend=True):
        x_=[]
        y_=[]
        for i in line:
            x_.append(i[0])
            y_.append(i[1])
            
        
        # fig=px.scatter(self.points)
        fig.add_trace(go.Scatter(x=x_,y=y_, name=name,showlegend=showlegend))
        return fig
        
    def add_point(self,point):
        if self.check_not_outside(point):
            self.points.append(point)
        
        
        

In [26]:
A=One_Line([1,1],1)
for i in range(-15,15):
    A.add_point([3,i])
A.show_spline()

-1.0 3.0


In [16]:
print(A.pmin)

-1.0


In [12]:
for i in range(-15,15):
    print(i,A.check_not_outside([3,i]))

-15 False
-14 False
-13 False
-12 False
-11 False
-10 False
-9 False
-8 False
-7 False
-6 False
-5 False
-4 False
-3 False
-2 False
-1 True
0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 False
9 False
10 False
11 False
12 False
13 False
14 False
